In [1]:
from requests import get
from bs4 import BeautifulSoup

In [6]:
def get_N_E(input_str):
    src_str = 'https://www.google.com/search?q=latitude+longitude+of+'
    loc_str = input_str.replace(' ', '+')
    tot_str = src_str + loc_str
    print(tot_str)
    response = get(tot_str)
    soup = BeautifulSoup(response.text, 'html.parser')
    a = soup.find('div', class_="BNeawe iBp4i AP7Wnd")
    string = a.text
    latitude, longitude = string.split(',')
    latitude_value, latitude_dir = latitude.strip().split(' ')
    longitude_value, longitude_dir = longitude.strip().split(' ')
    latitude_value = float(latitude_value[:-1])
    longitude_value = float(longitude_value[:-1])

    # Determine the latitude direction
    if latitude_dir == 'N':
        latitude_value = abs(latitude_value)
    else:
        latitude_value = -abs(latitude_value)
    # Determine the longitude direction
    if longitude_dir == 'E':
        longitude_value = abs(longitude_value)
    else:
        longitude_value = -abs(longitude_value)

    print(f"Latitude: {latitude_value}, Longitude: {longitude_value}")
    return latitude_value, longitude_value

In [7]:
import pandas as pd
import time

In [8]:
airport_df = pd.read_csv('Big_Airports.csv', header=None)
airport_df.columns = ['Airport_Name', 'Abriviation', 'City', 'Country', 'Traffic']
airport_df.head()

,Airport_Name,Abriviation,City,Country,Traffic
0,Hartsfield–Jackson Atlanta International Airport,ATL,Atlanta,United States,"103,902,992"
1,Los Angeles International Airport,LAX,Los Angeles,United States,"84,557,968"
2,O'Hare International Airport,ORD,Chicago,United States,"79,828,183"
3,Dallas-Fort Worth International Airport,DFW,Dallas,United States,"67,092,194"
4,Denver International Airport,DEN,Denver,United States,"61,379,396"


In [ ]:
longitude = []
latitude = []

for i in range(len(airport_df)):
    promt = airport_df['Airport_Name'][i] + '+' + airport_df['Abriviation'][i] + '+' + airport_df['City'][i] + '+' + airport_df['Country'][i]
    print(promt)
    lat, lon = get_N_E(promt)
    latitude.append(lat)
    longitude.append(lon)
    # wait for 1 second
    time.sleep(1)


In [11]:
# write the data into the dataframe
airport_df['Latitude'] = latitude
airport_df['Longitude'] = longitude

In [12]:
airport_df.head()

,Airport_Name,Abriviation,City,Country,Traffic,Latitude,Longitude
0,Hartsfield–Jackson Atlanta International Airport,ATL,Atlanta,United States,"103,902,992",33.6407,-84.4277
1,Los Angeles International Airport,LAX,Los Angeles,United States,"84,557,968",33.9416,-118.4085
2,O'Hare International Airport,ORD,Chicago,United States,"79,828,183",41.9803,-87.9090
3,Dallas-Fort Worth International Airport,DFW,Dallas,United States,"67,092,194",32.8998,-97.0403
4,Denver International Airport,DEN,Denver,United States,"61,379,396",39.8564,-104.6764


In [13]:
# save the dataframe into a csv file
airport_df.to_csv('Airport_location.csv', index=False)